In [11]:
""" This module implements the base function to implement a VQE for a Ising Chain. """
import pennylane as qml
from pennylane import numpy as np
import jax
import jax.numpy as jnp
from jax.example_libraries import optimizers

import copy
from tqdm.auto import tqdm
import pickle  # Writing and loading

import warnings

warnings.filterwarnings(
    "ignore",
    message="For Hamiltonians, the eigenvalues will be computed numerically. This may be computationally intensive for a large number of wires.Consider using a sparse representation of the Hamiltonian with qml.SparseHamiltonian.",
)

import PhaseEstimation.circuits as circuits
import PhaseEstimation.losses as losses
import PhaseEstimation.general as qmlgen

from typing import List, Set, Dict, Tuple, Optional
from numbers import Number
##############

def circuit_ising(N : int, params : List[Number]) -> int:
    """
    Full VQE circuit
    Number of parameters (gates): 7*N
    Parameters
    ----------
    N : int
        Number of qubits
    params: np.ndarray
        Array of parameters/rotation for the circuit

    Returns
    -------
    int
        Total number of parameters needed to build this circuit
    """
    # No wire will be measured until the end, the array of active
    # wire will correspond to np.arange(N) throughout the whole circuit
    active_wires = np.arange(N)
    index = 0
    qml.Barrier()
    for _ in range(6): # Depth of the circuit
        # Iterate circuit_ID9, the deeper the merrier
        index = circuits.circuit_ID9(active_wires, params, index)
        qml.Barrier()
    
    # Final independent rotations RX for each wire
    index = circuits.wall_gate(active_wires, qml.RX, params, index)
    
    return index


def circuit_ising2(N, params):
    """
    Full VQE circuit, enhanced version of circuit_ising, higher number of parameters
    Number of parameters (gates): 11*N

    Parameters
    ----------
    N : int
        Number of qubits
    params: np.ndarray
        Array of parameters/rotation for the circuit

    Returns
    -------
    int
        Total number of parameters needed to build this circuit
    """
    # No wire will be measured until the end, the array of active
    # wire will correspont to np.arange(N) throughout the whole circuit
    active_wires = np.arange(N)
    index = 0
    qml.Barrier()
    for _ in range(9):
        index = circuits.circuit_ID9(active_wires, params, index)
        qml.Barrier()
        
    index = circuits.wall_gate(active_wires, qml.RX, params, index)
    index = circuits.wall_gate(active_wires, qml.RY, params, index)
    
    return index

class vqe:
    def __init__(self, Hs, circuit):
        """
        Class for the VQE algorithm

        Parameters
        ----------
        Hs : class
            Custom Hamiltonian class
        circuit : function
            Function of the VQE circuit
        """
        self.Hs = Hs
        self.circuit = lambda p: circuit(self.Hs.N, p)
        self.circuit_fun = circuit
        # Pass the parameter array [0]*10000 (intentionally large) to the circuit
        # which it will output `index`, namely the number of parameters
        self.n_params = self.circuit([0] * 10000)
        # Initialize randomly all the parameter-arrays for each state
        self.vqe_params0 = jnp.array( np.random.uniform(-np.pi, np.pi, size=(self.Hs.n_states,self.n_params)) )
        self.device = qml.device("default.qubit.jax", wires=self.Hs.N, shots=None)
        
    def __repr__(self):
        # QCircuit just for printing it
        @qml.qnode(self.device, interface="jax")
        def vqe_state(self):
            self.circuit(np.arange(self.n_params))

            return qml.state()

        return qml.draw(vqe_state)(self)

In [18]:
from inspect import getmro
getmro(qml.CNOT)

(pennylane.ops.qubit.non_parametric_ops.CNOT,
 pennylane.operation.Operation,
 pennylane.operation.Operator,
 abc.ABC,
 object)

In [13]:
type(qml.RX)

abc.ABCMeta

In [7]:
import hamiltonians as hamiltonians
import annni_model as annni_model

In [10]:
type(Hs.qml_Hs[0])

pennylane.ops.qubit.hamiltonian.Hamiltonian

In [8]:
Hs = hamiltonians.hamiltonian(annni_model.build_Hs, N = 6, n_states = 10, ring = False)

In [29]:
myvqe = vqe(Hs, circuit_ising2)

In [30]:
print(myvqe)

0: ──||──H─╭C──RY(0.00)──────────────────────────────────────────||──H─╭C──RY(6.00)──────────
1: ──||──H─╰X─╭C─────────RY(1.00)────────────────────────────────||──H─╰X─╭C─────────RY(7.00)
2: ──||──H────╰X────────╭C─────────RY(2.00)──────────────────────||──H────╰X────────╭C───────
3: ──||──H──────────────╰X────────╭C─────────RY(3.00)────────────||──H──────────────╰X───────
4: ──||──H────────────────────────╰X────────╭C─────────RY(4.00)──||──H───────────────────────
5: ──||──H──────────────────────────────────╰X─────────RY(5.00)──||──H───────────────────────

──────────────────────────────────||──H─╭C──RY(12.00)────────────────────────────────────────────
──────────────────────────────────||──H─╰X─╭C──────────RY(13.00)─────────────────────────────────
───RY(8.00)───────────────────────||──H────╰X─────────╭C──────────RY(14.00)──────────────────────
──╭C─────────RY(9.00)─────────────||──H───────────────╰X─────────╭C──────────RY(15.00)───────────
──╰X────────╭C─────────RY(10.00)──||──H────